<a href="https://colab.research.google.com/github/JanhviSharma1/twitter-sentiment-analysis/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing kaggle
!pip install kaggle

In [7]:
#configure the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
#API to fetch the dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [9]:
#extracting the compressed data file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [10]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
#printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Data Processing

In [13]:
# loading the data from csv file to pandas dataframe
twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [14]:
twitter_data.shape

(1599999, 6)

In [15]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [16]:
#naming the columns and reading the dataset again
twitter_data.columns=['target','ids','date','flag','user','text']
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [17]:
def stemming(content):
  port_stem = PorterStemmer()
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [18]:
twitter_data['stemming_data'] = twitter_data['text'].apply(stemming)

In [19]:
twitter_data.head()

,target,ids,date,flag,user,text,stemming_data
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [20]:
print(twitter_data['stemming_data'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemming_data, Length: 1599999, dtype: object


In [32]:
#checking missing values
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0
stemming_data,0


In [34]:
#checking the distribution of target column
twitter_data['target'].value_counts()
twitter_data.replace({4:1},inplace=True)

In [35]:
twitter_data.sample(4)

,target,ids,date,flag,user,text,stemming_data
51753,0,1679132644,Sat May 02 08:36:50 PDT 2009,NO_QUERY,emunderpants,"I wanted to go and see the bunnies today, no s...",want go see bunni today luck
218540,0,1976193830,Sat May 30 16:47:32 PDT 2009,NO_QUERY,damanihiggins,I Wish I was with the Panos Family grilling.....,wish pano famili grill
828458,1,1557019127,Sun Apr 19 01:33:35 PDT 2009,NO_QUERY,djkboogie,@award562 Say wut now?,award say wut
848191,1,1564756790,Mon Apr 20 04:48:58 PDT 2009,NO_QUERY,coliwilso,"@meganrogers awww, that's lovely however, i t...",meganrog awww love howev think actual sleep


In [36]:
#separating data and label
X = twitter_data['stemming_data'].values
Y = twitter_data['target'].values

In [37]:
print(X)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [38]:
print(Y)

[0 0 0 ... 1 1 1]


# Splitting the data to training and test data

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [40]:
print(X.shape, X_train.shape, X_test.shape)

(1599999,) (1279999,) (320000,)


In [41]:
print(X_train)

['watch saw iv drink lil wine'
 'hire anoth employe gourmet point current hurri'
 'punish know much work tomorrow everyon els get day' ...
 'ohjustjak awkward crush nph amazingg'
 'oliveandfig wow never tweet nah yet tuesday spoil'
 'girl jenn stay twitter b c got back late amp tri relax enough sleep work amp awak']


In [42]:
print(X_test)

['school someon slap acrosss face caus stay class total dragggg poor poor kitti'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'dad comput turn even though weird mean download song want' ...
 'destini nevertheless hooray member wonder safe trip'
 'strawberri heavi appar broke blender epic fuck fail' 'supersandro thank']


In [43]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [44]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453002 stored elements and shape (1279999, 461471)>
  Coords	Values
  (0, 109298)	0.37470892870977707
  (0, 185125)	0.5296934860731303
  (0, 234920)	0.4196620311217523
  (0, 354566)	0.358082129508793
  (0, 436655)	0.27247657460069113
  (0, 443035)	0.44770025018080273
  (1, 19762)	0.25286013616648123
  (1, 86730)	0.33015139495167095
  (1, 119689)	0.4309123985667578
  (1, 152015)	0.49196931362728563
  (1, 166826)	0.3977368489124923
  (1, 172343)	0.3785321144028949
  (1, 319721)	0.3128279791558143
  (2, 93826)	0.2233049434503089
  (2, 117914)	0.3986115507964782
  (2, 124728)	0.33175938915396974
  (2, 146003)	0.2213048673720909
  (2, 220710)	0.2626149875630723
  (2, 279017)	0.2849974993622606
  (2, 326288)	0.5853283798404599
  (2, 412554)	0.28961919890983556
  (2, 445635)	0.23563700521896036
  (3, 46285)	0.6887696820070169
  (3, 93826)	0.46202690964703447
  (3, 331805)	0.4656145237310222
  :	:
  (1279996, 85239)	0.3425039810672

In [45]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289342 stored elements and shape (320000, 461471)>
  Coords	Values
  (0, 63768)	0.26422682589785035
  (0, 75261)	0.26552138110472423
  (0, 127038)	0.2730995033465573
  (0, 219133)	0.3295455088887744
  (0, 320624)	0.5238287544743887
  (0, 355868)	0.22363572756759686
  (0, 371332)	0.3972681545090477
  (0, 375613)	0.24658637821166524
  (0, 382367)	0.2541115193458825
  (0, 414033)	0.2624690535388268
  (1, 6452)	0.3061987253241848
  (1, 15098)	0.21044041574811612
  (1, 145408)	0.5774478727480551
  (1, 217431)	0.40260807391422926
  (1, 256686)	0.2861349119561766
  (1, 348128)	0.4737202797635419
  (1, 366259)	0.24522188688615762
  (2, 79341)	0.34512546063620464
  (2, 88422)	0.33529803711277506
  (2, 107866)	0.36857211925047617
  (2, 124443)	0.2729825984376057
  (2, 258609)	0.3072614985829275
  (2, 375890)	0.3020734075258265
  (2, 407333)	0.27060796399382786
  (2, 418804)	0.33257344872506
  :	:
  (319995, 193560)	0.373116119782695


# Train the Machine Learning Model
Logistic Regression

In [46]:
model = LogisticRegression(max_iter=1000)

In [47]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

# Model Evaluation
Accuracy Score

In [48]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [49]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.801063125830567


In [50]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [51]:
print("Accuracy score of the test data : ", test_data_accuracy)

Accuracy score of the test data :  0.7771625


# Saving the trained Model

In [52]:
import pickle